In [1]:
# 一、入门
# 了解TensorFlow的工作原理
# 主要是TensorFlow的三个模型:计算模型、数据模型、运行模型

In [2]:
import tensorflow as tf

In [3]:
# 基本使用 - 计算
a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([3.0, 4.0], name='b')
c = a + b

sess = tf.Session()
sess.run(c)
# sess.close()

array([4., 6.], dtype=float32)

In [4]:
# TensorFlow的模型:计算模型、数据模型、运行模型

In [5]:
# 1.计算模型 - 计算图
# 查看默认计算图
print(a.graph is tf.get_default_graph())

# 不同计算图上的张量和运算都不会共享
g1 = tf.Graph()
with g1.as_default():
    v = tf.get_variable('v', shape=[2,3], initializer=tf.zeros_initializer)
    
g2 = tf.Graph()
with g2.as_default():
    v = tf.get_variable('v', shape=[2,3], initializer=tf.ones_initializer)

# 在计算图g1中读取变量'v'
with tf.Session(graph=g1) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse=True):
        print(sess.run(tf.get_variable('v')))

# 在计算图g2中读取变量'v'
with tf.Session(graph=g2) as sess:
    tf.global_variables_initializer().run()
    with tf.variable_scope('', reuse=True):
        print(sess.run(tf.get_variable('v')))

True
[[0. 0. 0.]
 [0. 0. 0.]]
[[1. 1. 1.]
 [1. 1. 1.]]


In [6]:
# 2.数据模型 - 张量
# 张量保存的是计算过程，并非数字
a = tf.constant([1.0, 2.0], name='a')
b = tf.constant([3.0, 4.0], name='b', dtype=tf.float32)
c1 = a + b    # 计算的变量类型'dtype'必须一致
c2 = tf.add(a, b, name='addx')
c3 = tf.constant([1.0, 2.0], name='a') + tf.constant([3.0, 4.0], name='b')
print(c1)    # 'add:0'：命名'node:src_output'，说明'c'这个张量是计算节点'add'输出的第一个结果
print(c2)
print(c3)
print(tf.Session().run(c1))    # 得到计算结果

# 张量 - 可变的
# 注:TensorFlow中存在集合(collection)的概念,所有的变量都会自动被加入到GraphKeys.VARIABLES这个集合中
#    可以通过tf.global_variables()这个函数拿到当前计算图上的所有变量
#    当构建机器学习模型(如神经网络)时,可以通过变量声明函数的trainable参数来区分需要优化的参数(默认为True)
#    可以通过tf.trainable_variables函数得到所有需要优化的参数
#    TensorFlow的神经网络优化算法会将GraphKeys.VARIABLES集合中的变量作为默认优化的对象
# 故:不必手动实现反向传播,只需写好前向传播,TensorFlow会自动实现反向传播,且优化参数
x = tf.Variable([1,2,3], name='weight', dtype=tf.int32)

Tensor("add_1:0", shape=(2,), dtype=float32)
Tensor("addx:0", shape=(2,), dtype=float32)
Tensor("add_2:0", shape=(2,), dtype=float32)
[4. 6.]


In [7]:
# 3.运行模型 - 会话
# 会话拥有并管理TensorFlow程序运行时的所有支援
# 所有计算完后，需要关闭会话，本次使用到的资源可以被释放，否则就可能出现资源泄露问题
c = tf.constant([1.0, 2.0], name='a') + tf.constant([3.0, 4.0], name='b')
# 使用方式1 - 手动
sess = tf.Session()
sess.run(c)
sess.close()
# 使用方式2 - 自动
with tf.Session() as sess:
    print(sess.run(c))

[4. 6.]


In [8]:
# 指定为默认会话后，可以用tf.Tensor.eval()函数计算张量取值
c = tf.constant([1.0, 2.0], name='a') + tf.constant([3.0, 4.0], name='b')
sess = tf.Session()

with sess.as_default():
    print(c.eval())

print(c.eval(session=sess))    # 功能同上，但必须手动指定会话

# 附:(为了方便交互式环境下的使用，TensorFlow提供了直接构建默认会话的函数)
isess = tf.InteractiveSession()
print(c.eval())
isess.close()
# print(c.eval())    # error, don't appoint session

[4. 6.]
[4. 6.]
[4. 6.]


In [9]:
# 配置会话(Session或InteractiveSession皆可)
# ConfigProto可以配置并行线程数、GPU分配策略、运算超时时间等
# allow_soft_placement:允许在不支持GPU的设备上使用CPU,而不是报错
# log_device_placement:记录每个节点被安排在那个设备上，设为False可以减少日志量
config = tf.ConfigProto(allow_soft_placement=True, log_device_placement=True)
sess1 = tf.Session(config=config)
sess2 = tf.InteractiveSession(config=config)